# Введение в обработку текста на естественном языке

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Введение в обработку текста на естественном языке"
* https://www.nltk.org/api/nltk.metrics.distance.html
* https://pymorphy2.readthedocs.io/en/stable/user/guide.html
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

In [1]:
path = 'C://Users//icom1//OneDrive - icom//Рабочий стол//TOBD_datasets//'

## Лабораторная работа 7

1\. Загрузите данные из файла `ru_recipes_sample.csv` в виде `pd.DataFrame` `recipes` Используя регулярные выражения, удалите из описаний (столбец `description`) все символы, кроме русских букв, цифр и пробелов. Приведите все слова в описании к нижнему регистру. Сохраните полученный результат в столбец `description`.

In [2]:
import pandas as pd

In [3]:
recipes = pd.read_csv(path+'ru_recipes_sample.csv')
recipes.head(1)

,url,name,ingredients,description
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...",Этот коктейль готовлю из замороженной клубники...


In [4]:
import re
pattern = re.compile(r"(?:[^а-я0-9 ])", re.I)

In [5]:
pd.set_option('display.max_colwidth', None)
recipes.description = recipes.description.str.lower().replace(pattern, '')
recipes.description[0]

'этот коктейль готовлю из замороженной клубники если клубника свежая то добавляю перепелиное яйцо благодаря этому коктейль получается устойчиво густым'

### Расстояние редактирования

2\. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние Левенштейна. Выведите на экран результат в следующем виде:

```
d(word1, word2) = x
```

In [6]:
from nltk.tokenize import word_tokenize
words_ununique = recipes.description.apply(lambda i: word_tokenize(i))

In [7]:
words = []
for list_of_words in words_ununique:
    for word in list_of_words:
        if word in words:
            pass
        else:
            words.append(word)

In [8]:
from random import sample
lst_of_samples = [sample(words,2) for i in range(5)]
lst_of_samples

[['той', 'необычен'],
 ['превратится', 'закаток'],
 ['праздничный', 'настоялся'],
 ['взвесила', 'сырые'],
 ['такую', 'трудиться']]

In [9]:
from nltk import edit_distance
for i in range(5):
    print(f'd({lst_of_samples[i][0]}, {lst_of_samples[i][1]}) = {edit_distance(lst_of_samples[i][0],lst_of_samples[i][1])}')

d(той, необычен) = 7
d(превратится, закаток) = 9
d(праздничный, настоялся) = 10
d(взвесила, сырые) = 8
d(такую, трудиться) = 8


3\. Напишите функцию, которая принимает на вход 2 текстовые строки `s1` и `s2` и при помощи расстояния Левенштейна определяет, является ли строка `s2` плагиатом `s1`. Функция должна реализовывать следующую логику: для каждого слова `w1` из `s1` проверяет, есть в `s2` хотя бы одно слово `w2`, такое, что расстояние Левенштейна между `w1` и `w2` меньше 2, и считает количество таких слов в `s1` $P$. 

$$ P = \#\{w_1 \in s_1\ | \exists w_2 \in s_2 : d(w_1, w_2) < tol\}$$

$$ L = max(|s1|, |s2|) $$

Здесь $|\cdot|$ - количество слов в строке, $\#A$ - число элементов в множестве $A$, $w \in s$ означает, что слово $w$ содержится в тексте $s$.

Если отношение $P / L$ больше 0.8, то функция должна вернуть True; иначе False.

Продемонстрируйте работу вашей функции на примере описаний двух рецептов с ID 135488 и 851934 (ID рецепта - это число, стоящее в конце url рецепта). Выведите на экран описания этих рецептов и результат работы функции.

In [10]:
def is_plagiarism(s1: str, s2: str) -> bool:
    P = 0
    L = max(len(s1.split()),len(s2.split()))
    for w1 in s1.split():
        for w2 in s2.split():
            if edit_distance(w1, w2) < 2:
                P += 1
    return P/L > 0.8

In [11]:
s1 = recipes[recipes.url.str.contains('135488')]['description'].values[0]
s2 = recipes[recipes.url.str.contains('851934')]['description'].values[0]
print(s1 + '\n\n' + s2)
is_plagiarism(s1, s2)

прекрасной закуской к крепким напиткам на фуршетном столе станет паштет из сала и авокадо с чесноком который мы спрячем в орешках из бородинского хлеба положив в середину маслину закуска получается необычной оригинальной и ценится особенно мужчинами

замечательной закуской к напиткам на фуршетном столе станет паштет из сала и авокадо с чесноком куда мы добавим орешков из бородинского хлеба положив в середину маслины закуска получается крайне необычной оригинальной и ценится особенно мужчинами


True

### Стемминг, лемматизация

4\. На основе набора слов из задания 2 создайте `pd.DataFrame` со столбцами `word`, `stemmed_word` и `normalized_word`. В столбец `stemmed_word` поместите версию слова после проведения процедуры стемминга; в столбец `normalized_word` поместите версию слова после проведения процедуры лемматизации. Столбец `word` укажите в качестве индекса. 

Для стемминга можно воспользоваться `SnowballStemmer` из `nltk`, для лемматизации слов - пакетом `pymorphy2`. Сравните результаты стемминга и лемматизации. Поясните на примере одной из строк получившегося фрейма (в виде текстового комментария), в чем разница между двумя этими подходами. 

In [12]:
import numpy as np
from nltk.stem import SnowballStemmer
import pymorphy2

stemmer = SnowballStemmer("russian")
{w: stemmer.stem(w) for w in words}
stemmer = SnowballStemmer("russian")
morph = pymorphy2.MorphAnalyzer()
stem_df = pd.DataFrame()
stem_df['stemmed_word'] = [stemmer.stem(word) for word in words]
stem_df['normalized_word'] = [morph.parse(word)[0].normalized.word for word in words]
stem_df['word'] = words
stem_df.set_index('word')

,stemmed_word,normalized_word
word,,
этот,этот,этот
коктейль,коктейл,коктейль
готовлю,готовл,готовить
из,из,из
замороженной,заморожен,заморозить
...,...,...
сбывается,сбыва,сбываться
подружим,подруж,подружить
одинаковых,одинаков,одинаковый


*Стемминг - приведение слова к его минимальному корню (исключая приставки, окончания, суффиксы)*
---
***
*Лемматизация - приведение слова к начальной форме (слово осмысленно, просто в нулевой форме)*
---

5\. Добавьте в таблицу `recipes` столбец `description_no_stopwords`, в котором содержится текст описания рецепта после удаления из него стоп-слов. Посчитайте и выведите на экран долю стоп-слов среди общего количества слов. Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [13]:
from nltk.corpus import stopwords
def remove_stopwords(description):
    text_tokens = word_tokenize(description)
    st_words = stopwords.words('russian')
    lst_no_stopwords = [word for word in text_tokens if not word in st_words]
    return ' '.join(lst_no_stopwords)

In [14]:
recipes['description_no_stopwords'] = recipes.description.apply(lambda i: remove_stopwords(i))

In [15]:
recipes.description[0], recipes['description_no_stopwords'][0]

('этот коктейль готовлю из замороженной клубники если клубника свежая то добавляю перепелиное яйцо благодаря этому коктейль получается устойчиво густым',
 'коктейль готовлю замороженной клубники клубника свежая добавляю перепелиное яйцо благодаря этому коктейль получается устойчиво густым')

In [26]:
round(
    sum(
        [len(item) for item in recipes.description_no_stopwords]) / sum([len(item) for item in recipes.description]
        ),
    2
) * 100

83.0

In [94]:
words = {}
for list_of_words in words_ununique:
    for word in list_of_words:
        if word in words.keys():
            words[word] +=1
        else:
            words[word] = 1
from collections import Counter
Counter(words).most_common(10)

[('и', 5043),
 ('в', 2567),
 ('с', 1932),
 ('на', 1642),
 ('очень', 1594),
 ('не', 1520),
 ('из', 1005),
 ('я', 972),
 ('а', 850),
 ('рецепт', 843)]

In [102]:
no_stop_words = {}
for list_of_words in recipes['description_no_stopwords'].apply(lambda i: word_tokenize(i)).to_list():
    for word in list_of_words:
        if word in no_stop_words.keys():
            no_stop_words[word] +=1
        else:
            no_stop_words[word] = 1
from collections import Counter
Counter(no_stop_words).most_common(10)

[('очень', 1594),
 ('рецепт', 843),
 ('это', 728),
 ('блюдо', 521),
 ('вкусный', 459),
 ('просто', 434),
 ('вкусно', 366),
 ('приготовить', 342),
 ('вкус', 319),
 ('салат', 312)]

Без стоп слов в самых популярных появились смысловые слова, вместо различных предлогов, приставок и прочех частей речи не несущих информацию
---

### Векторное представление текста

6\. Выберите случайным образом 5 рецептов из набора данных, в названии которых есть слово "оладьи" (без учета регистра). Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`. На основе полученных векторов создайте `pd.DataFrame`, в котором названия колонок соответствуют словам из словаря объекта-векторизатора. 

Примечание: обратите внимание на порядок слов при создании колонок.

In [127]:
from sklearn.feature_extraction.text import TfidfVectorizer
sample_descr_list = recipes[recipes.name.str.lower().str.contains('оладьи')]['description'].sample(5).to_list()
corpus = sample_descr_list
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [128]:
vectors_df = pd.DataFrame.sparse.from_spmatrix(data = X, columns = vectorizer.get_feature_names_out())
vectors_df

,31,ароматные,базилика,без,блендер,более,бы,вам,версию,весной,...,только,томатные,угощайтесь,упростился,хозяйственник,чашу,щепотка,эльзара,это,яблочками
0,0.000000,0.227615,0.000000,0.227615,0.000000,0.000000,0.000000,0.000000,0.000000,0.227615,...,0.183638,0.000000,0.227615,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.144564,0.000000,0.000000,0.000000,0.144564,0.144564,0.144564,0.000000,...,0.116633,0.144564,0.000000,0.000000,0.144564,0.000000,0.144564,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.266317,0.000000,0.266317
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.260375,0.000000
4,0.125037,0.000000,0.000000,0.000000,0.125037,0.125037,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.125037,0.000000,0.125037,0.000000,0.000000,0.000000,0.000000


7\. Вычислите близость между каждой парой рецептов, выбранных в задании 6, используя косинусное расстояние (можно воспользоваться функциями из любого пакета: `scipy`, `scikit-learn` или реализовать функцию самому). Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

Примечание: обратите внимание, что $d_{cosine}(x, x) = 0$

In [129]:
names = []
for desc in sample_descr_list:
    name = recipes[recipes.description == desc]['name'].to_list()
    names.append(name[0])

In [130]:
from sklearn.metrics.pairwise import cosine_distances
df = pd.DataFrame(data = cosine_distances(vectors_df), columns = names, index = names)
df

,Оладьи с рубленым яйцом и луком,Оладьи из помидоров с мятой,Оладьи от Эльзары,Оладьи овсяные дрожжевые,Оладьи из печени
Оладьи с рубленым яйцом и луком,0.000000,0.978582,0.960543,1.000000,1.000000
Оладьи из помидоров с мятой,0.978582,0.000000,0.982732,0.958617,0.936722
Оладьи от Эльзары,0.960543,0.982732,0.000000,1.000000,0.985065
Оладьи овсяные дрожжевые,1.000000,0.958617,1.000000,0.000000,0.964207
Оладьи из печени,1.000000,0.936722,0.985065,0.964207,0.000000


8\. Напишите функцию, которая принимает на вход `pd.DataFrame`, полученный в задании 7, и возвращает в виде кортежа названия двух различных рецептов, которые являются наиболее похожими. Прокомментируйте результат (в виде текстового комментария). Для объяснения результата сравните слова в описаниях двух этих отзывов.

In [131]:
def find_closest(sim_df: pd.DataFrame) -> tuple:
    first = df[df>0].min().sort_values().index[0]
    second = df[df>0].min().sort_values().index[1]
    return (first, second)
        

In [132]:
find_closest(df)

('Оладьи из помидоров с мятой', 'Оладьи из печени')

In [133]:
recipes[recipes.name == find_closest(df)[0]]['description'].values[0]

'какойто хозяйственник проснулся в мом муже когда он принс домой несколько килограммов помидоров  теперь у нас дома начинаются томатные дни оладьи готовила полностью по рецепту от себя только щепотка базилика который как мне кажется помидорам всегда на пользу но я бы рецепт немного изменила выкладываю оригинальную версию со своими комментариями надеюсь вам понравится'

In [134]:
recipes[recipes.name == find_closest(df)[1]]['description'].values[0]

'31 декабря рабочий день  времени на подготовку праздничного стола ооочень мало на помощь приходят рецепты которые можно приготовить заранее рецепт оладушек простой но их не стыдно подать на праздничный стол тем более несложный процесс приготовления упростился окончательно сейчас на смену мясорубке пришел блендер или кухонный комбайн в чашу которого можно сложить все ингредиенты одновременно вуаля и вкусная закуска готова'

Были выбраны именно эти строки, потому что угол между их векторами минимален, например, есть одинаковые слова, или похожие друг на друга (Этот угол получился таким после преобразования TF-IDF: вес некоторого слова пропорционален частоте употребления этого слова в строке и обратно пропорционален частоте употребления слова во всех остальных строках)
---